In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import streamlit as st

# Set up SQLite database
conn = sqlite3.connect("redbus.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        route_name TEXT,
        route_link TEXT,
        busname TEXT,
        bustype TEXT,
        departing_time TIME,
        duration TEXT,
        reaching_time TIME,
        star_rating FLOAT,
        price DECIMAL,
        seats_available INT
    );
""")

# Set up Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# Function to scrape bus routes
def scrape_bus_routes():
    url = "https://www.redbus.in/"
    driver.get(url)
    routes = driver.find_elements(By.XPATH, "//div[@class='route']")
    for route in routes:
        route_name = route.find_element(By.XPATH, ".//h2[@class='route-name']").text
        route_link = route.find_element(By.XPATH, ".//a[@class='route-link']").get_attribute("href")
        busname = route.find_element(By.XPATH, ".//span[@class='bus-name']").text
        bustype = route.find_element(By.XPATH, ".//span[@class='bus-type']").text
        departing_time = route.find_element(By.XPATH, ".//span[@class='departing-time']").text
        duration = route.find_element(By.XPATH, ".//span[@class='duration']").text
        reaching_time = route.find_element(By.XPATH, ".//span[@class='reaching-time']").text
        star_rating = route.find_element(By.XPATH, ".//span[@class='star-rating']").text
        price = route.find_element(By.XPATH, ".//span[@class='price']").text
        seats_available = route.find_element(By.XPATH, ".//span[@class='seats-available']").text
        cursor.execute("""
            INSERT INTO bus_routes (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available))
        conn.commit()

# Scrape bus routes
scrape_bus_routes()

# Streamlit app
st.title("Redbus Bus Routes")
st.write("Select a route to view details:")
routes = cursor.execute("SELECT route_name, route_link FROM bus_routes").fetchall()
for route in routes:
    st.write(route[0])
    st.write(route[1])
    if st.button("View Details"):
        route_details = cursor.execute("SELECT * FROM bus_routes WHERE route_name=?", (route[0],)).fetchone()
        st.write(route_details)

# Filters
st.sidebar.title("Filters")
bustype_filter = st.sidebar.selectbox("Bus Type", ["Sleeper", "Seater", "AC", "Non-AC"])
route_filter = st.sidebar.selectbox("Route", ["Hyderabad to Bangalore", "Bangalore to Hyderabad", "Chennai to Bangalore", "Bangalore to Chennai"])
price_range_filter = st.sidebar.slider("Price Range", 0, 1000)
star_rating_filter = st.sidebar.slider("Star Rating", 0, 5)
availability_filter = st.sidebar.selectbox("Availability", ["Available", "Not Available"])
